In [97]:
from selenium import webdriver
import pandas as pd
import time

### Get all states

In [98]:
driver = webdriver.Chrome(executable_path=r'C:\Program Files\chromedriver.exe')
time.sleep(5)

# This is used to store 2 letter state abbreviations
states = []

driver.get("https://www.aopa.org/destinations/airports/state")

# Get a list of all states
state_link = driver.find_elements_by_xpath('//a[@class="state-listing__state-link"]')

# Parse out the last two characters of each URL
for s in state_link:
    state_abbreviation = s.get_attribute("href")[-2:]
    states.append(state_abbreviation)
    
driver.close()

In [109]:
print(states)

['AL', 'AK', 'AS', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'GU', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VI', 'VA', 'WA', 'WV', 'WI', 'WY']


### Get all airports

In [100]:
# Find badge for each airport
def element(xpath, xpath_list):
    try:
        airport_name.find_element_by_xpath(xpath).text
        return xpath_list.append("Y")   
    except:
        return xpath_list.append("N")

In [101]:
driver = webdriver.Chrome(executable_path=r'C:\Program Files\chromedriver.exe')
time.sleep(3)

airports = []
private_use = []
unpaved = []
heliport = []
military = []
seaplane_base = []
closed_indefinitely = []
state_list = []

for state in states:
    driver.get("https://www.aopa.org/destinations/airports/state/" + state)
    time.sleep(5)

    # Get each airport row (which includes names and badges)
    airport_names = driver.find_elements_by_xpath('//div[@class = "state-airports__airport"]')

    for airport_name in airport_names:

        # Find airport names
        airport = airport_name.find_element_by_xpath('.//a[@class="state-airports__airport-link"]').text
        airports.append(airport)

        # Find private airports
        element('.//div[@class="badge badge--private-use badge--compact"]', private_use)

        # Find unpaved airports
        element('.//div[@class="badge badge--unpaved badge--compact"]', unpaved)

        # Find heliports
        element('.//div[@class="badge badge--heliport badge--compact"]', heliport)

        # Find military airports
        element('.//div[@class="badge badge--military badge--compact"]', military)

        # Find seaplane base
        element('.//div[@class="badge badge--seaport badge--compact"]', seaplane_base) 

        # Find closed airports
        element('.//div[@class="badge badge--closed-badge--compact"]', closed_indefinitely)
        
        # Append state
        state_list.append(state)

driver.close()

In [105]:
df = pd.DataFrame({
    "Airports": airports,
    "Private use": private_use,
    "Unpaved": unpaved,
    "Heliport": heliport,
    "Military use": military,
    "Seaplane base": seaplane_base,
    "Closed indefinitely": closed_indefinitely,
    "State": state_list
})
df.head()

,Airports,Private use,Unpaved,Heliport,Military use,Seaplane base,Closed indefinitely,State
0,00AL - Epps Airpark,Y,Y,N,N,N,N,AL
1,01AL - Ware Island,Y,Y,N,N,N,N,AL
2,02A - Chilton County Airport,N,N,N,N,N,N,AL
3,02AL - Bass Field,Y,Y,N,N,N,N,AL
4,03AL - Highland Medical Center,Y,Y,Y,N,N,N,AL


In [106]:
# Split airport names into code and name
df[["Abbr", "Airport name"]] = df["Airports"].str.split(" -", 1, expand = True)
df.drop(["Airports"], axis=1, inplace = True)

In [107]:
df["Abbr"] = df["Abbr"].str.strip()
df

,Private use,Unpaved,Heliport,Military use,Seaplane base,Closed indefinitely,State,Abbr,Airport name
0,Y,Y,N,N,N,N,AL,00AL,Epps Airpark
1,Y,Y,N,N,N,N,AL,01AL,Ware Island
2,N,N,N,N,N,N,AL,02A,Chilton County Airport
3,Y,Y,N,N,N,N,AL,02AL,Bass Field
4,Y,Y,Y,N,N,N,AL,03AL,Highland Medical Center
...,...,...,...,...,...,...,...,...,...
18773,Y,Y,N,N,N,N,WY,WY64,Lone Pine Flying Ranch
18774,Y,Y,N,N,N,N,WY,WY65,Madsen
18775,Y,Y,N,N,N,N,WY,WY66,Chamberlain Brothers Ranch
18776,Y,Y,Y,N,N,N,WY,WY67,South Lincoln Medical Center
